<a href="https://colab.research.google.com/github/shubhamundirwade/Area_of_shape/blob/main/prophet_forcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pystan==2.19.1.1 
!pip install fbprophet 
!pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 8.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
# Importing libraries

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = datetime.today().strftime("%Y-%m-%d")
start_date = '2015-01-01'
eth_df = yf.download("ETH-USD", start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-04,"$2,783.13","$2,956.69","$2,779.27","$2,940.64","$2,940.64",18186749944
2022-05-05,"$2,940.23","$2,948.96","$2,704.92","$2,749.21","$2,749.21",22642925048
2022-05-06,"$2,748.93","$2,754.84","$2,645.34","$2,694.98","$2,694.98",21027599270
2022-05-07,"$2,694.99","$2,696.65","$2,599.52","$2,636.09","$2,636.09",13369276367
2022-05-08,"$2,637.41","$2,637.85","$2,532.79","$2,539.34","$2,539.34",16978925568


In [5]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1642 entries, 2017-11-09 to 2022-05-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1642 non-null   float64
 1   High       1642 non-null   float64
 2   Low        1642 non-null   float64
 3   Close      1642 non-null   float64
 4   Adj Close  1642 non-null   float64
 5   Volume     1642 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 89.8 KB


In [7]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [8]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [10]:
eth_df.reset_index(inplace=True )
eth_df.columns

Index(['index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [15]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [17]:
df.tail()

,ds,y
1637,2022-05-04,"$2,783.13"
1638,2022-05-05,"$2,940.23"
1639,2022-05-06,"$2,748.93"
1640,2022-05-07,"$2,694.99"
1641,2022-05-08,"$2,637.41"


In [20]:
# Ploting the open price

x = df['ds']
y = df['y']

fig = go.Figure()

fig.add_trace(go.Scatter(x = x, y = y))

#Setting title for plot

fig.update_layout(
    title_text = "Time Series plot of Ethereum Open Price"
)

fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list(
                [
                 dict(count = 1, label = '1d', step = 'day', stepmode = 'backward'),
                 dict(count = 1, label = '1m', step = 'month', stepmode = 'backward'),
                 dict(count = 6, label = '6m', step = 'month', stepmode = 'backward'),
                 dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                 dict(count = 1, label = '1y', step = 'year', stepmode = 'backward'),
                 dict(step = 'all'),
                ]
            )
    ),
    rangeslider = dict(visible=True),
    type = 'date',
    )
)

In [21]:
m = Prophet(seasonality_mode='multiplicative')
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [22]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2002,2023-05-04
2003,2023-05-05
2004,2023-05-06
2005,2023-05-07
2006,2023-05-08


In [23]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2002,2023-05-04,"$3,215.14","$1,525.27","$4,982.99"
2003,2023-05-05,"$3,236.78","$1,553.79","$5,029.41"
2004,2023-05-06,"$3,251.74","$1,558.12","$5,089.50"
2005,2023-05-07,"$3,294.60","$1,502.63","$5,128.68"
2006,2023-05-08,"$3,344.36","$1,554.83","$5,200.55"


In [24]:
next_day = (datetime.today() + timedelta(days = 1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

3502.761149757439

In [25]:
plot_plotly(m,forecast)

In [26]:
plot_components_plotly(m,forecast)